# Importing the necessary libraries

In [42]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

# Request to get accessed to the html of website

In [43]:
url_list_cheese = "https://www.fromages.com/fr/encyclopedie"
html_data = requests.get(url_list_cheese).text

In [44]:
soup_list_cheese = BeautifulSoup(html_data,"html.parser")

# Get the necessary tag elements where the information is contained

In [45]:
list_of_letters = []
list_a_tags = []
soup_letter = soup_list_cheese.find_all('div',{'class':'lettre_unit'})
for element in soup_letter:
    list_of_letters.append(element.find_all('a'))

for letter in list_of_letters:
    for a_tag in letter:
        list_a_tags.append("https://www.fromages.com/" + a_tag.get("href"))

# Create different functions for each attribute you want to collect

In [46]:
import numpy as np

def get_nom_fromage(soup):
    soup_nom_fromage = soup.find_all('h1',{'class': 'fromage_detail_titre'})[0].text
    nom_fromage = [word for word in soup_nom_fromage.replace("\r\n","").replace("\n\n","").split(" ") if word != ""]
    if nom_fromage == []:
        return np.nan
    else:
        nom_fromage = ' '.join(nom_fromage)
        return nom_fromage

In [47]:
import numpy as np

def get_poids(soup):
    soup_poids = soup.find_all('div',{'class':'picto_poids'})[0].text
    poids = [word for word in soup_poids.replace("\r\n","").replace("\n\n","").replace("\n","").split(" ") if word != ""]
    if poids == []:
        return np.nan
    else:
        poids.pop(0)
        poids = ' '.join(poids)
        return poids

In [48]:
import numpy as np

def get_affinage(soup):
    soup_affinage = soup.find_all('div',{'class':'picto_affinage'})[0].text
    affinage = [word for word in soup_affinage.replace("\r\n","").replace("\n\n","").replace("\n","").split(" ") if word != ""]
    if affinage == []:
        return np.nan
    else:    
        affinage.pop(0)
        affinage = ' '.join(affinage)
        return affinage

In [49]:
import numpy as np

def get_type(soup):
    soup_type = soup.find_all('div',{'class':'picto_type'})[0].text
    type = [word for word in soup_type.replace("\r\n","").replace("\n\n","").replace("\n","").split(" ") if word != ""]
    if type == []:
        return np.nan
    else:
        type.pop(0)
        type = ' '.join(type)
        return type

In [50]:
import numpy as np

def get_lait(soup):
    soup_lait = soup.find_all('div',{'class':'picto_lait'})[0].text
    lait = [word for word in soup_lait.replace("\r\n","").replace("\n\n","").replace("\n","").split(" ") if word != ""]
    if lait == []:
        return np.nan
    else:
        lait = ' '.join(lait)
        return lait

In [51]:
import numpy as np

def get_pate(soup):
    soup_pate = soup.find_all('div',{'class':'picto_pate'})[0].text
    pate = [word for word in soup_pate.replace("\r\n","").replace("\n\n","").replace("\n","").split(" ") if word != ""]

    if pate == []:
        return np.nan
    else:
        pate.pop(0)
        pate = ' '.join(pate)
        return pate

In [52]:
import re
import numpy as np

def get_saison(soup):
    soup_saison = soup.find_all('div',{'class':'picto_saison'})[0].text
    saison = [word for word in soup_saison.replace("\r\n","").replace("\n\n","").replace("\n","").split(" ") if word != ""]

    if saison == []:
        return np.nan
    else:
        saison.pop(0)
        saison = ' '.join(saison)
        saison = re.sub(r"([A-Z])", r" \1", saison).split()
        saison = ' '.join(saison)
        return saison

In [53]:
import numpy as np

def get_vin(soup):
    soup_vin = soup.find_all('p',{'class':'fromage_detail_vins_liste'})

    if soup_vin == []:
        return np.nan
    else:
        vin = [word for word in soup_vin[0].text.replace("\r\n","").replace("\n\n","").replace("\n","").split(" ") if word != ""]
        vin = ' '.join(vin)
        return vin

In [54]:
import numpy as np

def get_mariage_fromage(soup):
    soup_mariage_fromage = soup.find_all('ul',{'class':'fromage_detail_vins_liste'})

    if soup_mariage_fromage == []:
        return np.nan
    else:
        mariage_fromage = [word for word in soup_mariage_fromage[0].text.replace("\r\n","").replace("\n\n","").replace("\n","").split(" ") if word != ""]
        mariage_fromage = ' '.join(mariage_fromage)
        return mariage_fromage

In [55]:
import numpy as np

def get_ingredients(soup):
    soup_ingredients = soup.find_all('div',{'class':'col-xs-12 font-light'})

    if soup_ingredients == []:
        return np.nan
    else:
        ingredients = [word for word in soup_ingredients[0].text.replace("\r\n","").replace("\n\n","").replace("\n","").replace(":","").split(" ") if word != ""]
        ingredients.pop(0)
        if ingredients.count("/") > 0:
            ingredients = ingredients[:ingredients.index("/")]
        ingredients = ' '.join(ingredients)
        return ingredients

# Create the dataframe

In [56]:
df_cheese = pd.DataFrame(columns=["nom","poids","affinage","type","type_lait","pate","saison","mariage_vin","mariage_fromage","ingredients"])

In [57]:
for url in list_a_tags:
    data = requests.get(url).text
    soup = BeautifulSoup(data,"html.parser")
    nom_fromage = get_nom_fromage(soup)
    poids = get_poids(soup)
    affinage = get_affinage(soup)
    type = get_type(soup)
    lait = get_lait(soup)
    pate = get_pate(soup)
    saison = get_saison(soup)
    vin = get_vin(soup)
    mariage_fromage = get_mariage_fromage(soup)
    ingredients = get_ingredients(soup)

    df_cheese = df_cheese.append(
        {"nom":nom_fromage,
         "poids":poids,
         "affinage":affinage,
         "type":type,
         "type_lait":lait,
         "pate":pate,
         "saison":saison,
         "mariage_vin":vin,
         "mariage_fromage":mariage_fromage,
         "ingredients":ingredients}
        ,ignore_index=True)
    
df_cheese

,nom,poids,affinage,type,type_lait,pate,saison,mariage_vin,mariage_fromage,ingredients
0,Abbaye de Tamié,"510 g (17,85 Ozs)",2 moisMinimum,à la pièce,Laitde vache,pressées non cuites,Printemps Automne Hiver,Vin rouge: Beaujolais-Villages (cépage: Gamay ...,NaN,"lait de vache, ferments lactiques, présure, sel"
1,A Casinca,400 g (14 Ozs),,à la coupe,Laitde chèvre,molles à croûte lavée,Printemps Eté Automne Hiver,grand vin corse blanc patrimonio,NaN,NaN
2,Abbaye de Timadeuc,200 g (7 Ozs),3 semaines minimum,à la coupe,Laitde vache,pressées non cuites,Printemps Eté Automne,NaN,NaN,NaN
3,Abbaye du Mont des Cats,200 g (7 Ozs),2 mois minimum,à la coupe,Laitde vache,pressées non cuites,Printemps Eté Automne,"Vin rouge: Graves (cépages: Merlot, Cabernet-S...",NaN,NaN
4,Affidelice au Chablis,"250 g (8,75 Ozs)",3 à 4 semaines,à la pièce,Laitde vache,molles à croûte lavée,Printemps Automne Hiver,Vin blanc très sec: Chablis (cépage: Chardonnay),NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
339,Valencay AOP,"220 g (7,7 Ozs)",Minimum 5 semaines,à la pièce,Laitde chèvre,molle,Printemps Eté Automne,"Vin blanc sec: Valençay (cépages: Sauvignon, C...",Sainte Maure de Touraine AOP,"Lait cru de chèvre, sel, ferments lactiques, p..."
340,Venaco,200 g (7 Ozs),2 moisMinimum,à la coupe,Laitde brebis,molles à croûte lavée,Printemps Eté Automne Hiver,NaN,NaN,NaN
341,Vieux Lille,"187 g (6,545 Ozs)",3 moisMinimum,à la coupe,Laitde vache,molles à croûte lavée,Printemps Eté Automne Hiver,Champagne,NaN,NaN
342,Voeu du Poitou,"150 g (5,25 Ozs)",Minimum 2 semaines,à la coupe,Laitde chèvre,molle,Printemps Eté Automne,NaN,NaN,NaN


In [59]:
df_cheese['type_lait'] = df_cheese['type_lait'].str.replace("Laitde ","  ").replace('Lait','vache')
df_cheese['saison'] = df_cheese.saison.apply(lambda x: x[0:len(x)].split(' '))

In [60]:
df_cheese

,nom,poids,affinage,type,type_lait,pate,saison,mariage_vin,mariage_fromage,ingredients
0,Abbaye de Tamié,"510 g (17,85 Ozs)",2 moisMinimum,à la pièce,vache,pressées non cuites,"[Printemps, Automne, Hiver]",Vin rouge: Beaujolais-Villages (cépage: Gamay ...,NaN,"lait de vache, ferments lactiques, présure, sel"
1,A Casinca,400 g (14 Ozs),,à la coupe,chèvre,molles à croûte lavée,"[Printemps, Eté, Automne, Hiver]",grand vin corse blanc patrimonio,NaN,NaN
2,Abbaye de Timadeuc,200 g (7 Ozs),3 semaines minimum,à la coupe,vache,pressées non cuites,"[Printemps, Eté, Automne]",NaN,NaN,NaN
3,Abbaye du Mont des Cats,200 g (7 Ozs),2 mois minimum,à la coupe,vache,pressées non cuites,"[Printemps, Eté, Automne]","Vin rouge: Graves (cépages: Merlot, Cabernet-S...",NaN,NaN
4,Affidelice au Chablis,"250 g (8,75 Ozs)",3 à 4 semaines,à la pièce,vache,molles à croûte lavée,"[Printemps, Automne, Hiver]",Vin blanc très sec: Chablis (cépage: Chardonnay),NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
339,Valencay AOP,"220 g (7,7 Ozs)",Minimum 5 semaines,à la pièce,chèvre,molle,"[Printemps, Eté, Automne]","Vin blanc sec: Valençay (cépages: Sauvignon, C...",Sainte Maure de Touraine AOP,"Lait cru de chèvre, sel, ferments lactiques, p..."
340,Venaco,200 g (7 Ozs),2 moisMinimum,à la coupe,brebis,molles à croûte lavée,"[Printemps, Eté, Automne, Hiver]",NaN,NaN,NaN
341,Vieux Lille,"187 g (6,545 Ozs)",3 moisMinimum,à la coupe,vache,molles à croûte lavée,"[Printemps, Eté, Automne, Hiver]",Champagne,NaN,NaN
342,Voeu du Poitou,"150 g (5,25 Ozs)",Minimum 2 semaines,à la coupe,chèvre,molle,"[Printemps, Eté, Automne]",NaN,NaN,NaN


In [61]:
df_cheese.isnull().sum()

nom                  0
poids                0
affinage             0
type                 0
type_lait            0
pate                 0
saison               0
mariage_vin        101
mariage_fromage    281
ingredients        216
dtype: int64

The following information should be added for each type of cheese: <br>
- Ingredients
- Wine & Cheese pairing
- Details about the flavor of the cheese (e.g., mild, strong, spicy, sweet, bitter, salty, etc.) can be very useful.
- More detailed information about the texture (e.g., hard, soft, crumbly, creamy, etc.) could help to make better recommendations.
- The geographical origin of a cheese can have a significant impact on its characteristics, and some users may have preferences for cheeses from certain regions or countries.
- Nutritional Information: Details like fat content, calorie count, protein content, etc., might be important to some users.
<br>

You can get the following information from this website: [https://www.quiveutdufromage.com/liste-des-fromages]

Change the following information:
- Features like weight (poids) might need to be normalized so they're on a similar scale to your other features.


In [62]:
# Using this link, we notice that there is a table containing most of the information that we wanted our cheese dataframe to have.
#We can just use the read_html function from pandas to get the table and then merge it with our dataframe.
url_list_cheese = "https://www.quiveutdufromage.com/liste-des-fromages"
table = pd.read_html(url_list_cheese)
additional_cheese_information = table[0]

In [63]:
additional_cheese_information

,Fromage,Type de lait Tout voir lait de vache lait de chèvre Lait de brebis lait de bufflonne Lait de brebis et de chèvre,Origine & régions Tout voir Bourgogne Franche Comté Auvergne-Rhône-Alpes Suisse Italie Nouvelle-Aquitaine Provence Alpes Côte d'azur Hauts de France Occitanie Pays de la Loire Grand-Est Île de France Centre-Val de Loire Angleterre Corse Espagne Normandie Pays-Bas Grèce Belgique Bretagne,Type de pâte Tout voir Pâte molle à croûte lavée Pâte pressée cuite Pâte fraîche Pâte pressée non cuite Pâte molle à croûte fleurie Fromages de chèvre Pâte persillée Pâte pressée Pâte molle,Goût Tout voir Fruité Doux mi-fort Intense Équilibré
0,Abbaye de Cîteaux,lait de vache,Bourgogne Franche Comté,Pâte molle à croûte lavée,Fruité
1,Abbaye de Tamié,lait de vache,Auvergne-Rhône-Alpes,Pâte molle à croûte lavée,Fruité
2,Abondance AOP,lait de vache,Auvergne-Rhône-Alpes,Pâte pressée cuite,Fruité
3,Apérivrais®,lait de vache,Auvergne-Rhône-Alpes,Pâte fraîche,Doux
4,Appenzeller,lait de vache,Suisse,Pâte pressée non cuite,Fruité
...,...,...,...,...,...
190,Trèfle du Perche,lait de chèvre,Centre-Val de Loire,Fromages de chèvre,Doux
191,Vacherin fribourgeois,lait de vache,Suisse,Pâte pressée non cuite,Fruité
192,Valençay AOP,lait de chèvre,Centre-Val de Loire,Fromages de chèvre,Doux
193,Vieux Pané®,lait de vache,Pays de la Loire,Pâte molle à croûte lavée,Fruité


In [64]:
#First we will change the columns of the dataframe
additional_cheese_information.columns = ['nom', 'type_lait', 'origine', 'type_pate', 'gout']

In [65]:
additional_cheese_information

,nom,type_lait,origine,type_pate,gout
0,Abbaye de Cîteaux,lait de vache,Bourgogne Franche Comté,Pâte molle à croûte lavée,Fruité
1,Abbaye de Tamié,lait de vache,Auvergne-Rhône-Alpes,Pâte molle à croûte lavée,Fruité
2,Abondance AOP,lait de vache,Auvergne-Rhône-Alpes,Pâte pressée cuite,Fruité
3,Apérivrais®,lait de vache,Auvergne-Rhône-Alpes,Pâte fraîche,Doux
4,Appenzeller,lait de vache,Suisse,Pâte pressée non cuite,Fruité
...,...,...,...,...,...
190,Trèfle du Perche,lait de chèvre,Centre-Val de Loire,Fromages de chèvre,Doux
191,Vacherin fribourgeois,lait de vache,Suisse,Pâte pressée non cuite,Fruité
192,Valençay AOP,lait de chèvre,Centre-Val de Loire,Fromages de chèvre,Doux
193,Vieux Pané®,lait de vache,Pays de la Loire,Pâte molle à croûte lavée,Fruité


In [67]:
#Now we will have to merge this new data with the rest of our dataframe df_cheese
#We must take into account that on this website there are cheeses that are not on the other website and vice versa
#We will have to merge the information from the two tables based on the name of the cheese
#We will use the merge function from pandas
#We will use the inner join method to keep all the information from both dataframes
#We will use the nom column to merge the two dataframes
df_cheese = pd.merge(df_cheese,additional_cheese_information,how="inner",on="nom")

#By merging both of the tables, we notice that there are only 35 cheeses that are in both tables with complete information
#We have to find another way to add new information on our dataframe

,nom,poids,affinage,type,type_lait_x,pate,saison,mariage_vin,mariage_fromage,ingredients,type_lait_y,origine_x,type_pate_x,gout_x,type_lait,origine_y,type_pate_y,gout_y
0,Abbaye de Tamié,"510 g (17,85 Ozs)",2 moisMinimum,à la pièce,vache,pressées non cuites,"[Printemps, Automne, Hiver]",Vin rouge: Beaujolais-Villages (cépage: Gamay ...,NaN,"lait de vache, ferments lactiques, présure, sel",lait de vache,Auvergne-Rhône-Alpes,Pâte molle à croûte lavée,Fruité,lait de vache,Auvergne-Rhône-Alpes,Pâte molle à croûte lavée,Fruité
1,Appenzeller,200 g (7 Ozs),3 mois minimum,à la coupe,vache,pressées cuites,"[Printemps, Eté, Automne, Hiver]",NaN,NaN,NaN,lait de vache,Suisse,Pâte pressée non cuite,Fruité,lait de vache,Suisse,Pâte pressée non cuite,Fruité
2,Banon AOP,"90 g (3,15 Ozs)",Frais:minimum 2 semainesSemi-Frais:minimum 4 à...,à la pièce,chèvre,molle,"[Printemps, Eté, Automne]","Cassis blanc Cépage: Ugni, Clairette, Marsanne...",NaN,"lait de chèvre, sel, ferments, chlorure de cal...",lait de chèvre,Provence Alpes Côte d'azur,Fromages de chèvre,mi-fort,lait de chèvre,Provence Alpes Côte d'azur,Fromages de chèvre,mi-fort
3,Beaufort AOP,200 g (7 Ozs),4 mois minimum,à la coupe,vache,pressées cuites,"[Printemps, Eté, Automne, Hiver]",Vin blanc: Roussette de Savoie (cépage: Altess...,Vouvray Moelleux Philippe Foreau 2003 Bôfavre,"Lait, sel, ferments, présure",lait de vache,Auvergne-Rhône-Alpes,Pâte pressée cuite,Fruité,lait de vache,Auvergne-Rhône-Alpes,Pâte pressée cuite,Fruité
4,Bethmale,200 g (7 Ozs),Plus de 60 jours,à la coupe,vache,pressées non cuites,"[Printemps, Eté, Automne, Hiver]","Vin rouge: Collioure (cépages: Grenache Noir, ...",NaN,"Lait, sel, ferments, chlorure de calcium, prés...",Lait de brebis,Occitanie,Pâte pressée non cuite,Fruité,Lait de brebis,Occitanie,Pâte pressée non cuite,Fruité
5,Bleu de Gex AOP,200 g (7 Ozs),2 mois minimum,à la coupe,vache,persillée,"[Printemps, Eté, Automne]",Vin rouge fruité et léger: Côtes-du-Jura (cépa...,NaN,NaN,lait de vache,Bourgogne Franche Comté,Pâte persillée,Doux,lait de vache,Bourgogne Franche Comté,Pâte persillée,Doux
6,Brie de Meaux AOP,200 g (7 Ozs),Minimum 8 semaines,à la coupe,vache,molle à croûte fleurie,"[Eté, Automne, Hiver]",Vin rouge fruité: Monthélie (cépage: Pinot Noi...,Coulommiers 1/2 pc Camembert de Normandie AOP,"lait cru de vache, ferments lactiques,présure,...",lait de vache,Île de France,Pâte molle à croûte fleurie,Fruité,lait de vache,Île de France,Pâte molle à croûte fleurie,Fruité
7,Brie de Melun AOP,200 g (7 Ozs),,à la pièce,vache,molle à croûte fleurie,"[Eté, Automne, Hiver]",St Emilion,NaN,"lait de vache, sel, ferments, présure, CACI2, ...",lait de vache,Centre-Val de Loire,Pâte molle à croûte fleurie,Intense,lait de vache,Centre-Val de Loire,Pâte molle à croûte fleurie,Intense
8,Cathare,200 g (7 Ozs),Minimum 2 semaines,à la pièce,chèvre,molle,[Eté],"Vin blanc: Gaillac (cépages: Mauzac, L'En de l...",NaN,NaN,lait de chèvre,Occitanie,Fromages de chèvre,Doux,lait de chèvre,Occitanie,Fromages de chèvre,Doux
9,Cabécou,"40 g (1,4 Ozs)",,à la pièce,chèvre,molle,"[Printemps, Eté, Automne]",Vin blanc: Sancerre (Sauvignon),Saint Marcellin IGP,NaN,lait de chèvre,Occitanie,Fromages de chèvre,Doux,lait de chèvre,Occitanie,Fromages de chèvre,Doux
